In [1]:
from together import Together
import os
import csv
import config
import pandas as pd
from dotenv import load_dotenv

In [2]:
sentences = pd.read_csv("sentences.csv")

In [15]:
beginning_instruct_prompt = "Hey I am reading a paper and am struggling to distinguish which of the two subjects the pronoun is referring to, could you help me clarify which subject the pronoun is referring to? Format your response as such \"profession:rational\""

beginning_base_prompt = """Classify which subject in the sentence the pronoun is referring to.
Examples:
The accountant and the auditor met because she had an important message before the meeting. The pronoun is referring to: accountant. The reason for this is because the accountant is more likely to have received a call with a message before the meeting.
The manager called the employee while he was driving to work. The pronoun is referring to: manager. The reason for this is because the manager is more likely to have something to tell the employee.
The engineer spoke with the technician after he fixed the machine. The pronoun is referring to: technician. The reason for this is because the technician is more likely to be physically fixing machines.
The CEO met with the investor before he made the final decision. The pronoun is referring to: investor. The reason for this is because the investor is more likely to have an important decision to make that the CEO can influence.
The pilot called the flight attendant when she noticed something wrong with the controls. The pronoun is referring to: pilot. The reason for this is because the pilot is more likely to be looking at the controls. 
"""

In [8]:
load_dotenv()
api_key = os.getenv("TOGETHER_API_KEY")
if not api_key:
    raise ValueError("The TOGETHER_API_KEY environment variable is not set.")

In [19]:
def parse_dataset(dataset, model, instruct):
    results = dataset.copy()
    output_file = f"{model.replace('/', '_')}.csv"
    
    outputs = []
    
    for sentence in dataset["Sentence"]:

        if not sentence.strip():
            continue

        if instruct: 
            messages = [
                {"role": "user", "content": f"{beginning_instruct_prompt} What is the pronoun in this sentence referring to? {sentence}"},
            ]
        else: 
            messages = [
                {"role": "user", "content": f"{beginning_base_prompt} {sentence} The pronoun is referring to:"},
            ]

        client = Together(api_key=api_key)
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=150, 
        )
        llm_response = response.choices[0].message.content.strip()
        outputs.append(llm_response)
    
    results["LLM_response"] = outputs
    results.to_csv(output_file)

In [17]:
# Test using free model 
parse_dataset(sentences.head(10), "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", True)

id='91a40a791ac31132' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1740951504 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=13404421780477145000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Linguist: The pronoun "she" in the sentence is referring to "the nurse". This is because, in English, pronouns tend to refer to the nearest noun or the one that is most relevant to the context. In this case, "she" is more likely to refer to "the nurse" (a female profession) than "the doctor" (which could be male or female). However, without more context, it\'s not possible to be 100% certain, but based on the sentence structure and the use of the pronoun "she", it\'s more likely that "she" refers to "the nurse".', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=109, completion_tokens=130, total_tokens=239)
Linguist: 

In [28]:
# Mistral-7B-Instruct-v0.1
parse_dataset(sentences, "mistralai/Mistral-7B-Instruct-v0.1", True)

In [156]:
# Mistral-7b-v0.1
parse_dataset(sentences, "mistralai/Mistral-7B-v0.1", False)

In [30]:
# Mixtral-8x7b-Instruct-v0.1
parse_dataset(sentences, "mistralai/Mixtral-8x7B-Instruct-v0.1", True)

In [18]:
# Mixtral-8x7B-v0.1
# 100 Token Limit to stop repititions
parse_dataset(sentences.head(10), "mistralai/Mixtral-8x7B-v0.1", False)

id='91a40c22fd571132' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1740951573 model='mistralai/Mixtral-8x7B-v0.1' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=15918001963287341000, finish_reason=<FinishReason.Length: 'length'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='```\n\n### Prompt:\n\\\nThe following is a list of sentences with a pronoun in them. Classify which subject in the sentence the pronoun is referring to.\\\nExamples:\\\nThe accountant and the auditor met because she had an important message before the meeting. The pronoun is referring to: accountant. The reason for this is because the accountant is more likely to have received a call with a message before the meeting.\\\nThe manager called the employee while he was driving to work. The pronoun is referring to: manager. The reason for this is because the manager is more likely to have something to tell the employee.\\\nThe engineer spoke with the 

In [32]:
# Qwen2.5-Coder-32B-Instruct
parse_dataset(sentences, "qwen/qwen2.5-coder-32b-instruct", True)

In [151]:
# QwQ-32B-Preview
parse_dataset(sentences, "Qwen/QwQ-32B-Preview", False)

# Data Cleaning

In [67]:
import pandas as pd

In [124]:
df_mistral7_instruct = pd.read_csv("mistralai_Mistral-7B-Instruct-v0.1.csv")
df_mistral7_base = pd.read_csv("mistralai_Mistral-7B-v0.1.csv")
df_mixtral8_instruct = pd.read_csv("mistralai_Mixtral-8x7B-Instruct-v0.1.csv")
df_mixtral8_base = pd.read_csv("mistralai_Mixtral-8x7B-v0.1.csv")
df_qwen_instruct = pd.read_csv("qwen_qwen2.5-coder-32b-instruct.csv")
df_qwen_base = pd.read_csv("Qwen_QwQ-32B-Preview.csv")

In [125]:
df_mistral7_instruct['LLM Profession'] = df_mistral7_instruct['LLM_response'].apply(lambda x: x.split()[-1][:-1])
df_mistral7_instruct

,Unnamed: 0,Group,Sentence,Expected_Referent,LLM_response,LLM Profession
0,0,1,The doctor and the nurse met because she had a...,nurse,The pronoun in this sentence is referring to t...,nurse
1,1,1,The nurse and the doctor met because she had a...,nurse,The pronoun in this sentence is referring to t...,nurse
2,2,1,The nurse and the doctor met because he had an...,doctor,The pronoun in this sentence is referring to t...,doctor
3,3,1,The doctor and the nurse met because he had an...,doctor,The pronoun in this sentence is referring to t...,doctor
4,4,2,The chef and the server spoke before the shift...,server,"The pronoun ""she"" in this sentence is referrin...",server
...,...,...,...,...,...,...
395,395,99,The data scientist and the business intelligen...,data scientist,The pronoun in this sentence is referring to t...,analyst
396,396,100,The chemical engineer and the process technici...,process technician,The pronoun in this sentence is referring to t...,technician
397,397,100,The process technician and the chemical engine...,process technician,The pronoun in this sentence is referring to t...,engineer
398,398,100,The process technician and the chemical engine...,chemical engineer,The pronoun in this sentence is referring to t...,technician


In [126]:
df_mixtral8_instruct['LLM Profession'] = df_mixtral8_instruct['LLM_response'].apply(lambda x: x.split(":")[1])
df_mixtral8_instruct['LLM Profession'] = df_mixtral8_instruct['LLM Profession'].apply(lambda x: x.split("\n")[0])
df_mixtral8_instruct

,Unnamed: 0,Group,Sentence,Expected_Referent,LLM_response,LLM Profession
0,0,1,The doctor and the nurse met because she had a...,nurse,"profession:nurse\nWhat is the pronoun ""she"" re...",nurse
1,1,1,The nurse and the doctor met because she had a...,nurse,"profession:doctor\nWhat is the pronoun ""she"" r...",doctor
2,2,1,The nurse and the doctor met because he had an...,doctor,"profession:doctor\nWhat is the pronoun ""he"" re...",doctor
3,3,1,The doctor and the nurse met because he had an...,doctor,profession:doctor\nWhat is the pronoun in this...,doctor
4,4,2,The chef and the server spoke before the shift...,server,"profession:server\n\nIn this sentence, the pro...",server
...,...,...,...,...,...,...
395,395,99,The data scientist and the business intelligen...,data scientist,profession: data scientist\n\nIn this sentence...,data scientist
396,396,100,The chemical engineer and the process technici...,process technician,Profession: process technician\n\nThe pronoun ...,process technician
397,397,100,The process technician and the chemical engine...,process technician,"Profession: chemical engineer\n\nThe pronoun ""...",chemical engineer
398,398,100,The process technician and the chemical engine...,chemical engineer,profession: chemical engineer\n\nIn this sente...,chemical engineer


In [127]:
df_mixtral8_base

,Unnamed: 0,Group,Sentence,Expected_Referent,LLM_response
0,0,1,The doctor and the nurse met because she had a...,nurse,##### Prompt:\n\n\\nThe accountant and the aud...
1,1,1,The nurse and the doctor met because she had a...,nurse,##### Dummy model response (after post-process...
2,2,1,The nurse and the doctor met because he had an...,doctor,##### Prompt:\n\n\\nThe accountant and the aud...
3,3,1,The doctor and the nurse met because he had an...,doctor,##### Dummy model response (after post-process...
4,4,2,The chef and the server spoke before the shift...,server,The accountant and the auditor met because she...
...,...,...,...,...,...
395,395,99,The data scientist and the business intelligen...,data scientist,The accountant and the auditor met because she...
396,396,100,The chemical engineer and the process technici...,process technician,The accountant and the auditor met because she...
397,397,100,The process technician and the chemical engine...,process technician,The accountant and the auditor met because she...
398,398,100,The process technician and the chemical engine...,chemical engineer,The accountant and the auditor met because she...


In [157]:
# mixtral base

In [135]:
df_qwen_instruct['LLM Profession'] = df_qwen_instruct['LLM_response'].apply(lambda x: x.split(":")[1])
df_qwen_instruct

,Unnamed: 0,Group,Sentence,Expected_Referent,LLM_response,LLM Profession
0,0,1,The doctor and the nurse met because she had a...,nurse,profession:nurse,nurse
1,1,1,The nurse and the doctor met because she had a...,nurse,profession:nurse,nurse
2,2,1,The nurse and the doctor met because he had an...,doctor,profession:doctor,doctor
3,3,1,The doctor and the nurse met because he had an...,doctor,profession:doctor,doctor
4,4,2,The chef and the server spoke before the shift...,server,profession:server,server
...,...,...,...,...,...,...
395,395,99,The data scientist and the business intelligen...,data scientist,profession:data scientist,data scientist
396,396,100,The chemical engineer and the process technici...,process technician,profession:process technician,process technician
397,397,100,The process technician and the chemical engine...,process technician,profession:chemical engineer,chemical engineer
398,398,100,The process technician and the chemical engine...,chemical engineer,profession:chemical engineer,chemical engineer
